# Imports

In [118]:
import fire
import sqlite3

from sklearn.feature_extraction.text import CountVectorizer

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import re
import pandas as pd

[nltk_data] Downloading package stopwords to /home/anmol/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/anmol/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/anmol/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Constants

In [119]:
DEFAULT_DB_RESULTS = '../../reddit-engagement-files/results.db'
FOLLOWED_SUBS = ['codcompetitive']

# Useful Functions

In [120]:
def preprocess_text(text):
    # remove urls/numbers
    text = str(text).lower()
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[0-9]+', '', text)
    # expand contractions
    text = re.sub(r"'ve'", ' have', text)
    text = re.sub(r"n't", ' not', text)
    # remove emojis and special characters
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    text = re.sub('[^a-zA-Z0-9]+', ' ', text)
    # remove stop words
    stops = stopwords.words('english')
    text = ' '.join([x for x in text.split() if x not in stops])
    # tokenize and lemmatize
    text = word_tokenize(text)
    lm = WordNetLemmatizer()
    text = [lm.lemmatize(x) for x in text]
    return text

In [121]:
def read_activity_batch(db_path, start, end):
    connection = None
    # exploring ways to ensure file not open elsewhere
    try:
        with open(db_path, 'r') as _:
            connection = sqlite3.connect(db_path)
    except IOError:
        print('In use / other issue accessing %s' % db_path)
    # sort by date and preserve other ordering, return subset
    select_all = ''' SELECT * FROM activity;'''
    df_batch = pd.read_sql_query(select_all, connection)
    df_batch.access_time = pd.to_datetime(df_batch.access_time)
    df_batch = df_batch.reset_index().sort_values(by=['access_time', 'index'])
    df_batch = df_batch[df_batch.subreddit.apply(lambda x: x.lower() not in FOLLOWED_SUBS)]
    df_batch = df_batch.drop(['index'], axis=1)
    return df_batch[start:end]

In [122]:
def nlp_on_batch(df_batch):
    # combine all X somehow
    # combine all Y somehow
    # countvectorize
    # binary classification
    # f1-score
    pass

# NLP For Binary Classification

In [4]:
import pandas as pd
df = pd.read_sql_query('SELECT * FROM activity;', conn)
print(len(df))
df.head(n=10)

1548


,id,title,subreddit,access_time,action
0,l9c06q,Avergage people are just as greedy as rich peo...,unpopularopinion,01/31/2021 20:39:56,down
1,l9bayq,Obese people infuriate me.,TrueOffMyChest,01/31/2021 20:39:56,down
2,l90oq6,I am proud to do my part in paying forward our...,wallstreetbets,01/31/2021 20:39:56,down
3,l8xsfa,"Cool, but why would they bother telling us?",wallstreetbets,01/31/2021 20:39:56,down
4,kzul59,There is absolutely no goddamn reason why you ...,unpopularopinion,01/31/2021 20:39:56,down
5,kwf3ub,Apple launches major new Racial Equity and Jus...,apple,01/31/2021 20:39:56,down
6,kurmrg,I wonder how many pros consider GAing pros?,CoDCompetitive,01/31/2021 20:39:56,down
7,ktf66r,Latinx is bullshit,TrueOffMyChest,01/31/2021 20:39:56,down
8,krk1c0,Why is it a taboo to criticize cultures that d...,TooAfraidToAsk,01/31/2021 20:39:56,down
9,krhja1,CMV: The white teen who said the n word on a S...,changemyview,01/31/2021 20:39:56,down


In [35]:
c = df.tail(n=10).copy()
c[c.subreddit.apply(lambda x: x.lower() not in ['codcompetitive'])]

,id,title,subreddit,access_time,action
1539,ldemd0,"Nope, not in the great US of A!",WhitePeopleTwitter,02/05/2021 21:00:04,up
1540,ldekzg,Steph and Ayesha Curry Have Quietly Served Up ...,UpliftingNews,02/05/2021 21:00:04,up
1541,ldbhtp,But but but the trickle,PoliticalHumor,02/05/2021 21:00:04,up
1542,ld8c35,Remaster of the Mass Effect games changes some...,SubredditDrama,02/05/2021 21:00:04,up


## Preprocessing

In [101]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
import re

[nltk_data] Downloading package punkt to /home/anmol/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/anmol/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/anmol/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [102]:
# preprocessor here
def preprocess_text(text):
    # remove urls/numbers
    text = str(text).lower()
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[0-9]+', '', text)
    # expand contractions
    text = re.sub(r"'ve'", ' have', text)
    text = re.sub(r"n't", ' not', text)
    # remove emojis and special characters
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    text = re.sub('[^a-zA-Z0-9]+', ' ', text)
    # remove stop words
    stops = stopwords.words('english')
    text = ' '.join([x for x in text.split() if x not in stops])
    # tokenize
    text = word_tokenize(text)
    lm = WordNetLemmatizer()
    text = [lm.lemmatize(x) for x in text]
    return text

In [115]:
df['new'] = df.title.apply(lambda x: preprocess_text(x))
df['vote'] = df.action.apply(lambda x: 0 if x=='down' else 1)
df

,id,title,subreddit,access_time,action,new,vote
0,l9c06q,Avergage people are just as greedy as rich peo...,unpopularopinion,01/31/2021 20:39:56,down,"[avergage, people, greedy, rich, people]",0
1,l9bayq,Obese people infuriate me.,TrueOffMyChest,01/31/2021 20:39:56,down,"[obese, people, infuriate]",0
2,l90oq6,I am proud to do my part in paying forward our...,wallstreetbets,01/31/2021 20:39:56,down,"[proud, part, paying, forward, good, fortune, ...",0
3,l8xsfa,"Cool, but why would they bother telling us?",wallstreetbets,01/31/2021 20:39:56,down,"[cool, would, bother, telling, u]",0
4,kzul59,There is absolutely no goddamn reason why you ...,unpopularopinion,01/31/2021 20:39:56,down,"[absolutely, goddamn, reason, kid]",0
...,...,...,...,...,...,...,...
1543,ldpl62,They should scrap the current CDL camos and ju...,CoDCompetitive,02/06/2021 01:00:04,up,"[scrap, current, cdl, camo, hire, vlionman, wo...",1
1544,ldrx5l,240p vs 1080p,CoDCompetitive,02/06/2021 10:00:03,up,"[p, v, p]",1
1545,ldsvnf,Karen6 is in shambles rn,CoDCompetitive,02/06/2021 10:00:03,up,"[karen, shamble, rn]",1
1546,ldv3wi,Petition to vLionMan make the CDL weapon skins...,CoDCompetitive,02/06/2021 10:00:03,up,"[petition, vlionman, make, cdl, weapon, skin, ...",1


In [117]:
Xy = df[['new', 'vote']]
Xy

,new,vote
0,"[avergage, people, greedy, rich, people]",0
1,"[obese, people, infuriate]",0
2,"[proud, part, paying, forward, good, fortune, ...",0
3,"[cool, would, bother, telling, u]",0
4,"[absolutely, goddamn, reason, kid]",0
...,...,...
1543,"[scrap, current, cdl, camo, hire, vlionman, wo...",1
1544,"[p, v, p]",1
1545,"[karen, shamble, rn]",1
1546,"[petition, vlionman, make, cdl, weapon, skin, ...",1


In [105]:
from sklearn.feature_extraction.text import CountVectorizer

In [106]:
cv_text = CountVectorizer("haven't heard of wouldn't")

In [107]:
cv_text

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8',
                input="haven't heard of wouldn't", lowercase=True, max_df=1.0,
                max_features=None, min_df=1, ngram_range=(1, 1),
                preprocessor=None, stop_words=None, strip_accents=None,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None,
                vocabulary=None)

In [110]:
cv_text.vocabulary_

AttributeError: 'CountVectorizer' object has no attribute 'vocabulary_'